In [2]:
# Impordid
from src.transformers.models.bert.tokenization_bert import BertTokenizer
from src.transformers.models.bert.modeling_bert import BertEmbeddings, BertForMaskedLM
import torch
from estnltk import Text
#from estnltk.corpus_processing.parse_enc import parse_enc_file_iterator
from src.transformers.models.bert.configuration_bert import BertConfig
from transformers import AdamW
from tqdm.auto import tqdm
import numpy as np

In [3]:
# Tokeniseerija
tokenizer = BertTokenizer(vocab_file = "vocab2.txt", vocab_file_form = "vocab_form.txt")

C:\Users\rauln\Documents\makatoo\transformers\src\transformers\tokenization_utils.py PreTrainedTokenizer __init__
C:\Users\rauln\Documents\makatoo\transformers\src\transformers\tokenization_utils_base.py PreTrainedTokenizerBase __init__
C:\Users\rauln\Documents\makatoo\transformers\src\transformers\tokenization_utils_base.py SpecialTokensMixin __init__
C:\Users\rauln\Documents\makatoo\transformers\src\transformers\tokenization_utils_base.py SpecialTokensMixin unk_token
C:\Users\rauln\Documents\makatoo\transformers\src\transformers\tokenization_utils_base.py SpecialTokensMixin sep_token
C:\Users\rauln\Documents\makatoo\transformers\src\transformers\tokenization_utils_base.py SpecialTokensMixin pad_token
C:\Users\rauln\Documents\makatoo\transformers\src\transformers\tokenization_utils_base.py SpecialTokensMixin cls_token
C:\Users\rauln\Documents\makatoo\transformers\src\transformers\tokenization_utils_base.py SpecialTokensMixin mask_token
C:\Users\rauln\Documents\makatoo\transformers\src

In [4]:
sisend = ["Väljas sajab."]

In [5]:
# Dataseti loomine dataloaderi jaoks
class Dataset(torch.utils.data.Dataset):
    def __init__(self, encodings):
        self.encodings = encodings
    def __len__(self):
        return self.encodings["input_ids"].shape[0]
    def __getitem__(self, i):
        return {key: tensor[i] for key, tensor in self.encodings.items()}

In [6]:
# Maskimine
def mlm(tensor):
    rand = torch.rand(tensor[:, :, 0].shape)
    mask_arr = (rand < 0.15) * (tensor[:, :, 0] > 5) # 15% tokenitest maskitakse, special tokeneid ei maskita (id vocabis < 5)
    for i in range(tensor[:, :, 0].shape[0]):
        selection = torch.flatten(mask_arr[i].nonzero()).tolist()
        tensor[i, selection] = 4 # [MASK] tokeni ID mõlemas vocabis
    return tensor

In [7]:
# Andmestiku ettevalmistamine treenimiseks või mudelis kasutamiseks
def prepare_data(data):
    input_ids = []
    mask = []
    labels = []
    
    # Tokeniseerijaga surume kõik laused sama pikaks, ehk kui lause on pikem kui max_length, siis lõikame lõpu maha,
    # kui lause on lühem kui max_length, siis lisame lause lõppu vajaliku arvu [PAD] tokeneid.
    sample = tokenizer(data, max_length = 8, padding = "max_length", truncation = True, return_tensors = "pt")
    
    
    labels.append(sample.input_ids[:, :, 1]) # Labeliks võtame sõnavormi id
    mask.append(sample.attention_mask) # Attention_mask märgistab [PAD] tokeneid (0 - [PAD], 1 - kõik muud tokenid)
    input_ids.append(mlm(sample.input_ids.detach().clone()))

    # Listist tensoriks
    input_ids = torch.cat(input_ids)
    mask = torch.cat(mask)
    labels = torch.cat(labels)

    encodings = {
    "input_ids" : input_ids,
    "attention_mask" : mask,
    "labels" : labels
    }
    
    dataset = Dataset(encodings)
    return dataset


# Dataloaderi loomine mudeli treenimiseks
dataloader_train = torch.utils.data.DataLoader(prepare_data(sisend), batch_size = 16, shuffle = True)

C:\Users\rauln\Documents\makatoo\transformers\src\transformers\tokenization_utils_base.py PreTrainedTokenizerBase __call__
C:\Users\rauln\Documents\makatoo\transformers\src\transformers\tokenization_utils_base.py PreTrainedTokenizerBase _switch_to_input_mode
C:\Users\rauln\Documents\makatoo\transformers\src\transformers\tokenization_utils_base.py PreTrainedTokenizerBase _call_one
C:\Users\rauln\Documents\makatoo\transformers\src\transformers\tokenization_utils_base.py PreTrainedTokenizerBase batch_encode_plus
C:\Users\rauln\Documents\makatoo\transformers\src\transformers\tokenization_utils_base.py PreTrainedTokenizerBase _get_padding_truncation_strategies
C:\Users\rauln\Documents\makatoo\transformers\src\transformers\tokenization_utils_base.py SpecialTokensMixin pad_token
C:\Users\rauln\Documents\makatoo\transformers\src\transformers\tokenization_utils_base.py SpecialTokensMixin pad_token_id
C:\Users\rauln\Documents\makatoo\transformers\src\transformers\tokenization_utils_base.py Speci

In [8]:
for batch in dataloader_train:
    print(batch)

{'input_ids': tensor([[[   2,    2],
         [1611,    1],
         [2114,   53],
         [   6,    1],
         [   3,    3],
         [   0,    0],
         [   0,    0],
         [   0,    0]]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 0, 0, 0]]), 'labels': tensor([[ 2,  1, 53,  1,  3,  0,  0,  0]])}


In [9]:
%%time
# Mudeli treenimine

# Config faili vaikeparameetrite üle kirjutamine
config = BertConfig(
    vocab_size = tokenizer.vocab_size,
    vocab_size_form = tokenizer.vocab_size_form,
    tie_word_embeddings = False
)

# Mudeli loomine
model = BertForMaskedLM(config)
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu") # Kui võimalik siis treenime GPU peal

model.to(device)
optim = AdamW(model.parameters(), lr = 1e-4)
epochs = 2

for epoch in range(epochs):
    loop = tqdm(dataloader_train, leave = True)
    for batch in loop:
        # clear the gradients
        optim.zero_grad()
        # Batchi andmed device'i
        input_ids = batch["input_ids"].to(device)
        mask = batch["attention_mask"].to(device)
        labels = batch["labels"].to(device)
        # compute the model output
        outputs = model(input_ids, attention_mask = mask, labels = labels)
        # obtain loss
        loss = outputs.loss
        # credit assignment
        loss.backward()
        # update model weights
        optim.step()

        loop.set_description(f"Epoch: {epoch}")
        loop.set_postfix(loss = loss.item())

C:\Users\rauln\Documents\makatoo\transformers\src\transformers\models\bert\tokenization_bert.py BertTokenizer vocab_size
C:\Users\rauln\Documents\makatoo\transformers\src\transformers\models\bert\tokenization_bert.py BertTokenizer vocab_size_form
C:\Users\rauln\Documents\makatoo\transformers\src\transformers\models\bert\configuration_bert.py BertConfig __init__
C:\Users\rauln\Documents\makatoo\transformers\src\transformers\models\bert\modeling_bert.py BertForMaskedLM __init__
C:\Users\rauln\Documents\makatoo\transformers\src\transformers\models\bert\modeling_bert.py BertModel __init__
C:\Users\rauln\Documents\makatoo\transformers\src\transformers\models\bert\modeling_bert.py BertEmbeddings __init__
C:\Users\rauln\Documents\makatoo\transformers\src\transformers\models\bert\modeling_bert.py BertEncoder __init__
C:\Users\rauln\Documents\makatoo\transformers\src\transformers\models\bert\modeling_bert.py BertLayer __init__
C:\Users\rauln\Documents\makatoo\transformers\src\transformers\model

C:\Users\rauln\Documents\makatoo\transformers\src\transformers\models\bert\modeling_bert.py BertPreTrainedModel _init_weights
C:\Users\rauln\Documents\makatoo\transformers\src\transformers\models\bert\modeling_bert.py BertPreTrainedModel _init_weights
C:\Users\rauln\Documents\makatoo\transformers\src\transformers\models\bert\modeling_bert.py BertPreTrainedModel _init_weights
C:\Users\rauln\Documents\makatoo\transformers\src\transformers\models\bert\modeling_bert.py BertPreTrainedModel _init_weights
C:\Users\rauln\Documents\makatoo\transformers\src\transformers\models\bert\modeling_bert.py BertPreTrainedModel _init_weights
C:\Users\rauln\Documents\makatoo\transformers\src\transformers\models\bert\modeling_bert.py BertPreTrainedModel _init_weights
C:\Users\rauln\Documents\makatoo\transformers\src\transformers\models\bert\modeling_bert.py BertPreTrainedModel _init_weights
C:\Users\rauln\Documents\makatoo\transformers\src\transformers\models\bert\modeling_bert.py BertPreTrainedModel _init_

C:\Users\rauln\Documents\makatoo\transformers\src\transformers\models\bert\modeling_bert.py BertPreTrainedModel _init_weights
C:\Users\rauln\Documents\makatoo\transformers\src\transformers\models\bert\modeling_bert.py BertPreTrainedModel _init_weights
C:\Users\rauln\Documents\makatoo\transformers\src\transformers\models\bert\modeling_bert.py BertPreTrainedModel _init_weights
C:\Users\rauln\Documents\makatoo\transformers\src\transformers\models\bert\modeling_bert.py BertPreTrainedModel _init_weights
C:\Users\rauln\Documents\makatoo\transformers\src\transformers\models\bert\modeling_bert.py BertPreTrainedModel _init_weights
C:\Users\rauln\Documents\makatoo\transformers\src\transformers\models\bert\modeling_bert.py BertPreTrainedModel _init_weights
C:\Users\rauln\Documents\makatoo\transformers\src\transformers\models\bert\modeling_bert.py BertPreTrainedModel _init_weights
C:\Users\rauln\Documents\makatoo\transformers\src\transformers\models\bert\modeling_bert.py BertPreTrainedModel _init_

C:\Users\rauln\Documents\makatoo\transformers\src\transformers\models\bert\modeling_bert.py BertPreTrainedModel _init_weights
C:\Users\rauln\Documents\makatoo\transformers\src\transformers\models\bert\modeling_bert.py BertPreTrainedModel _init_weights
C:\Users\rauln\Documents\makatoo\transformers\src\transformers\models\bert\modeling_bert.py BertPreTrainedModel _init_weights
C:\Users\rauln\Documents\makatoo\transformers\src\transformers\models\bert\modeling_bert.py BertPreTrainedModel _init_weights
C:\Users\rauln\Documents\makatoo\transformers\src\transformers\models\bert\modeling_bert.py BertPreTrainedModel _init_weights
C:\Users\rauln\Documents\makatoo\transformers\src\transformers\models\bert\modeling_bert.py BertPreTrainedModel _init_weights
C:\Users\rauln\Documents\makatoo\transformers\src\transformers\models\bert\modeling_bert.py BertPreTrainedModel _init_weights
C:\Users\rauln\Documents\makatoo\transformers\src\transformers\models\bert\modeling_bert.py BertPreTrainedModel _init_

C:\Users\rauln\Documents\makatoo\transformers\src\transformers\models\bert\modeling_bert.py BertPreTrainedModel _init_weights
C:\Users\rauln\Documents\makatoo\transformers\src\transformers\models\bert\modeling_bert.py BertPreTrainedModel _init_weights
C:\Users\rauln\Documents\makatoo\transformers\src\transformers\models\bert\modeling_bert.py BertPreTrainedModel _init_weights
C:\Users\rauln\Documents\makatoo\transformers\src\transformers\models\bert\modeling_bert.py BertPreTrainedModel _init_weights
C:\Users\rauln\Documents\makatoo\transformers\src\transformers\models\bert\modeling_bert.py BertPreTrainedModel _init_weights
C:\Users\rauln\Documents\makatoo\transformers\src\transformers\models\bert\modeling_bert.py BertPreTrainedModel _init_weights
C:\Users\rauln\Documents\makatoo\transformers\src\transformers\models\bert\modeling_bert.py BertPreTrainedModel _init_weights
C:\Users\rauln\Documents\makatoo\transformers\src\transformers\models\bert\modeling_bert.py BertPreTrainedModel _init_

...

  0%|          | 0/1 [00:00<?, ?it/s]

C:\Users\rauln\Documents\makatoo\transformers\src\transformers\models\bert\modeling_bert.py BertForMaskedLM forward
C:\Users\rauln\Documents\makatoo\transformers\src\transformers\models\bert\modeling_bert.py BertModel forward
C:\Users\rauln\Documents\makatoo\transformers\src\transformers\models\bert\modeling_bert.py BertEmbeddings forward
C:\Users\rauln\Documents\makatoo\transformers\src\transformers\models\bert\modeling_bert.py BertEncoder forward
C:\Users\rauln\Documents\makatoo\transformers\src\transformers\models\bert\modeling_bert.py BertLayer forward
C:\Users\rauln\Documents\makatoo\transformers\src\transformers\models\bert\modeling_bert.py BertAttention forward
C:\Users\rauln\Documents\makatoo\transformers\src\transformers\models\bert\modeling_bert.py BertSelfAttention forward
C:\Users\rauln\Documents\makatoo\transformers\src\transformers\models\bert\modeling_bert.py BertSelfAttention transpose_for_scores
C:\Users\rauln\Documents\makatoo\transformers\src\transformers\models\bert

  0%|          | 0/1 [00:00<?, ?it/s]

C:\Users\rauln\Documents\makatoo\transformers\src\transformers\models\bert\modeling_bert.py BertForMaskedLM forward
C:\Users\rauln\Documents\makatoo\transformers\src\transformers\models\bert\modeling_bert.py BertModel forward
C:\Users\rauln\Documents\makatoo\transformers\src\transformers\models\bert\modeling_bert.py BertEmbeddings forward
C:\Users\rauln\Documents\makatoo\transformers\src\transformers\models\bert\modeling_bert.py BertEncoder forward
C:\Users\rauln\Documents\makatoo\transformers\src\transformers\models\bert\modeling_bert.py BertLayer forward
C:\Users\rauln\Documents\makatoo\transformers\src\transformers\models\bert\modeling_bert.py BertAttention forward
C:\Users\rauln\Documents\makatoo\transformers\src\transformers\models\bert\modeling_bert.py BertSelfAttention forward
C:\Users\rauln\Documents\makatoo\transformers\src\transformers\models\bert\modeling_bert.py BertSelfAttention transpose_for_scores
C:\Users\rauln\Documents\makatoo\transformers\src\transformers\models\bert

CPU times: total: 18 s
Wall time: 3.84 s


In [10]:
# Rakendamine
# Sisend
inp = tokenizer("Väljas [MASK].",return_tensors='pt')
# Asukohad, kus on [MASK] token
mask_loc = set(np.where(inp.input_ids.numpy()[0] == 4)[0].tolist())
# Mudeli väljund
out = model(inp["input_ids"]).logits[0].detach().numpy()
for masked in mask_loc:
    # Kõige suurem väärtus [MASK] tokeni väljundile
    predicted_token = np.argmax(out[masked]).tolist()
    decoded_token = tokenizer.decode(predicted_token, return_form = True)
    print(decoded_token)

C:\Users\rauln\Documents\makatoo\transformers\src\transformers\tokenization_utils_base.py PreTrainedTokenizerBase __call__
C:\Users\rauln\Documents\makatoo\transformers\src\transformers\tokenization_utils_base.py PreTrainedTokenizerBase _switch_to_input_mode
C:\Users\rauln\Documents\makatoo\transformers\src\transformers\tokenization_utils_base.py PreTrainedTokenizerBase _call_one
C:\Users\rauln\Documents\makatoo\transformers\src\transformers\tokenization_utils_base.py PreTrainedTokenizerBase encode_plus
C:\Users\rauln\Documents\makatoo\transformers\src\transformers\tokenization_utils_base.py PreTrainedTokenizerBase _get_padding_truncation_strategies
C:\Users\rauln\Documents\makatoo\transformers\src\transformers\tokenization_utils.py PreTrainedTokenizer _encode_plus
C:\Users\rauln\Documents\makatoo\transformers\src\transformers\tokenization_utils.py PreTrainedTokenizer tokenize
C:\Users\rauln\Documents\makatoo\transformers\src\transformers\tokenization_utils_base.py SpecialTokensMixin a

In [11]:
model(inp["input_ids"]).logits[0].size()

C:\Users\rauln\Documents\makatoo\transformers\src\transformers\tokenization_utils_base.py BatchEncoding __getitem__
C:\Users\rauln\Documents\makatoo\transformers\src\transformers\models\bert\modeling_bert.py BertForMaskedLM forward
C:\Users\rauln\Documents\makatoo\transformers\src\transformers\models\bert\modeling_bert.py BertModel forward
C:\Users\rauln\Documents\makatoo\transformers\src\transformers\models\bert\modeling_bert.py BertEmbeddings forward
C:\Users\rauln\Documents\makatoo\transformers\src\transformers\models\bert\modeling_bert.py BertEncoder forward
C:\Users\rauln\Documents\makatoo\transformers\src\transformers\models\bert\modeling_bert.py BertLayer forward
C:\Users\rauln\Documents\makatoo\transformers\src\transformers\models\bert\modeling_bert.py BertAttention forward
C:\Users\rauln\Documents\makatoo\transformers\src\transformers\models\bert\modeling_bert.py BertSelfAttention forward
C:\Users\rauln\Documents\makatoo\transformers\src\transformers\models\bert\modeling_bert.

torch.Size([5, 50005])

In [12]:
#tokenizer.decode(batch["input_ids"][0], return_form = True)
tokenizer.decode(batch["input_ids"][0])

C:\Users\rauln\Documents\makatoo\transformers\src\transformers\tokenization_utils_base.py PreTrainedTokenizerBase decode
C:\Users\rauln\Documents\makatoo\transformers\src\transformers\tokenization_utils.py PreTrainedTokenizer _decode
C:\Users\rauln\Documents\makatoo\transformers\src\transformers\tokenization_utils.py PreTrainedTokenizer convert_ids_to_tokens
C:\Users\rauln\Documents\makatoo\transformers\src\transformers\models\bert\tokenization_bert.py BertTokenizer _convert_id_to_token
C:\Users\rauln\Documents\makatoo\transformers\src\transformers\tokenization_utils_base.py SpecialTokensMixin unk_token
C:\Users\rauln\Documents\makatoo\transformers\src\transformers\models\bert\tokenization_bert.py BertTokenizer _convert_id_to_token
C:\Users\rauln\Documents\makatoo\transformers\src\transformers\tokenization_utils_base.py SpecialTokensMixin unk_token
C:\Users\rauln\Documents\makatoo\transformers\src\transformers\models\bert\tokenization_bert.py BertTokenizer _convert_id_to_token
C:\Users

'[CLS] väljas sadama. [SEP] [PAD] [PAD] [PAD]'